In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution.Gnuplot;
using BoSSS.Solution.Control;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// Is used at Jenkins to generate individual names (for output .json)
//string database_name = "DB_XdgPoisson";
string database_name = "trash";
string buildname = System.Environment.GetEnvironmentVariable("BUILD_DISPLAY_NAME");
string buildNr = System.Environment.GetEnvironmentVariable("BUILD_NUMBER");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "run" : buildname;
string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
buildNr = String.IsNullOrEmpty(buildNr)? thedate : buildNr;
string table_name = String.Concat(buildname, "_", buildNr);

In [ ]:
//table_name="run_21-12-2021"

run_21-12-2021

In [ ]:
string CurrentDocDir = Directory.GetCurrentDirectory();
string path = String.Concat(CurrentDocDir,@"\",table_name,".json");
var SubTab = TableExtensions.LoadFromFile(path);

In [ ]:
SubTab.GetColumnNames().OrderBy(s=>s);

In [ ]:
SubTab.Rows.Count

1

In [ ]:
var tab                        = SubTab.ExtractColumns("LinearSolver.SolverCode", "DOFs","DGdegree:u","maxSolRunT","ProjectName","SessionName","NoIter");
System.Data.DataView dv        = tab.DefaultView;
dv.Sort                        = "LinearSolver.SolverCode,DOFs";
System.Data.DataTable sortedDT = dv.ToTable();
sortedDT.Print();

   LinearSolver.SolverCode DOFs    DGdegree:u maxSolRunT  ProjectName    SessionName                      NoIter 
0: 41                      2076460 2          251.3953873 run_21-12-2021 J103823_k2_exp_Kcycle_schwarz_c4 26     

In [ ]:
public static class DoWhatUWant{

    public static PlotRowSelector Solver_DG_Proj(int Degree){
           return delegate (int iTabRow, 
                     IDictionary<string, object> Row, 
                     out string Nmn, 
                     out PlotFormat Fmt){
    
               Nmn = "";
               Fmt = new PlotFormat();
                         
               
              
               LinearSolverCode SolverCode = (LinearSolverCode)Convert.ToInt32(Row["LinearSolver.SolverCode"]);
               int DGdegree   = Convert.ToInt32(Row["DGdegree:u"]);
               string PrjName = Convert.ToString(Row["ProjectName"]);
    
               switch(SolverCode){
                   case LinearSolverCode.exp_Kcycle_schwarz:
                       Nmn+="Kcycle w. add.-Schwarz";
                       //Fmt.LineColor = LineColors.Blue;
                       Fmt.PointType = PointTypes.LowerTriangle;
                       break;
                   case LinearSolverCode.exp_gmres_levelpmg:
                       Nmn+="GMRES w. pTG";
                       //Fmt.LineColor = LineColors.Yellow;
                       Fmt.PointType = PointTypes.Box;
                       break;
                   case LinearSolverCode.classic_pardiso:
                       Nmn+="Pardiso";
                       //Fmt.LineColor = LineColors.Red;
                       Fmt.PointType = PointTypes.OpenCircle;
                       Fmt.DashType  = DashTypes.Dotted;
                       break;
                   default:
                       Nmn = null;
                       Fmt = null;
                    return;
               }
                
               
               if(DGdegree!=Degree){
                    Nmn = null;
                    Fmt = null;
                    return;
               }
               switch(DGdegree){
                   case 2:
                     Nmn+=" DG2";
                     //Fmt.PointType = PointTypes.OpenDiamond;
                     break;
                   case 3:
                     Nmn+=" DG3";
                     //Fmt.PointType = PointTypes.OpenDiamond;
                     break;
                   case 5:
                     Nmn+=" DG5";
                     //Fmt.PointType = PointTypes.OpenDiamond;
                     break;
    
                   default:
                     Nmn = null;
                     Fmt = null;
                     return;
               }
    //           switch(PrjName){
    //               case "SIP_weakscale":
    //                Nmn +=" openmp";
    //                Fmt.LineColor = LineColors.Blue;
    //                break;
    //               case "SIP_weakscale_SEQ":
    //                Nmn += " seq";
    //                Fmt.LineColor = LineColors.Yellow;
    //                break;
    //               default:
    //                Nmn = null;
    //                Fmt = null;
    //                return;
    //           }
                 
               Fmt.PointSize = 0.5;
               Fmt.LineWidth = 3;
               Fmt.Style     = Styles.LinesPoints;
               Fmt.LineColor = LineColors.Black;
               //Fmt.DashType  = DashTypes.Solid;
           };
        
    }
    
    public static PlotRowSelectorEx Solver_DG_Proj(int DG, int Solver){
               return delegate (int iSweep, int iTabRow, 
                     IDictionary<string, object> Row, 
                     out string Nmn, 
                     out PlotFormat Fmt,
                     out double xValue, out double yValue) { 
               // - - - - - - - - - - - - - - - - - - - - - - - - 
               // PlotRowSelector:
               // selects, which table row goes to which graph,
               // and the respective color  
               // - - - - - - - - - - - - - - - - - - - - - - - - 
               int k           = Convert.ToInt32(Row["DGdegree:u"]);
               int solver_name = Convert.ToInt32(Row["LinearSolver.SolverCode"]);
              
               if(k != DG || solver_name != Solver) {
                    // degree does not match -> not in this plot
                    Nmn    = null;
                    Fmt    = null;
                    xValue = 0;
                    yValue = 0;
                    return;
               }
               
               double mtxAssi  = Convert.ToDouble(Row["XdgMatrixAssembly"]);
               double aggBinit = Convert.ToDouble(Row["Aggregation_basis_init_time"]);
               double slvInit  = Convert.ToDouble(Row["Solver_Init_time"]);
               double slvRun   = Convert.ToDouble(Row["Solver_Run_time"]);
               double cores    = Convert.ToDouble(Row["NoOfCores"]);
               
               if(aggBinit > 700)
               Console.Write("aggBinit/slvRun = " + aggBinit + "/" + slvRun);
               
               Fmt           = new PlotFormat();
               Fmt.PointSize = 0.5;
               Fmt.Style     = Styles.LinesPoints;
               Fmt.LineColor = LineColors.Black;
               Fmt.LineWidth = 3;
               
               xValue = cores;
               
               switch(iSweep) {
                   case 0:
                   yValue       = slvRun;
                   Nmn          = "Slv Iter";
                   Fmt.DashType = DashTypes.Dotted;
                   Fmt.PointType = PointTypes.OpenCircle;
                   break;
                   
                   case 1:
                   yValue        = slvInit;
                   Nmn           = "Slv Init";
                   Fmt.DashType  = DashTypes.Dashed;
                   Fmt.PointType = PointTypes.OpenBox;
                   break;
                   
                   case 2:
                   yValue        = aggBinit;
                   Nmn           = "Agg Init";
                   Fmt.DashType  = DashTypes.DotDashed;
                   Fmt.PointType = PointTypes.OpenDiamond;
                   break;
                   
                   case 3:
                   yValue        = mtxAssi;
                   Nmn           = "Mtx ass";
                   Fmt.DashType  = DashTypes.DotDotDashed;
                   Fmt.PointType = PointTypes.OpenUpperTriangle;
                   break;
                   
                   default:
                   throw new NotImplementedException();
               }
               
               if(aggBinit > 700)
               Console.WriteLine("\t + case: {0} + yValue = {1}", iSweep, yValue);
           };
    }
    
      
       
    }

In [ ]:
// ==========
// parameterz
// ==========
int DG   = 2;
int xMin = 4;
int xMax = 512;
// ======================
// print runtime vs cores
// ======================
Plot2Ddata[,] multiplots = new Plot2Ddata[2,1];
multiplots[0,0] = SubTab.ToPlot("NoOfCores","maxSolRunT",DoWhatUWant.Solver_DG_Proj(DG));
multiplots[0,0].LogX = true;
multiplots[0,0].LogY = true;
multiplots[0,0].tmargin = 0;
multiplots[0,0].bmargin = 2;
multiplots[0,0].XrangeMin=xMin;
multiplots[0,0].XrangeMax=xMax;
multiplots[0,0].Ylabel = "~wallclock time";
multiplots[0,0].LegendAlignment=new string[]{"i","b","r"};
// ==============
// print 2h limit
// ==============
double[] xaxis = new[]{(double)xMin,(double)xMax};
double[] T = xaxis.Select(x => 7200.0).ToArray();
var linT              = new Plot2Ddata.XYvalues("2h limit", xaxis, T);
linT.Format.FromString("- black");
linT.Format.LineWidth = 2;
linT.Format.DashType  = DashTypes.DotDashed;
ArrayTools.AddToArray(linT, ref multiplots[0,0].dataGroups);
// =====================
// print noiter vs cores
// =====================
multiplots[1,0] = SubTab.ToPlot("NoOfCores","NoIter",DoWhatUWant.Solver_DG_Proj(DG));
multiplots[1,0].LogX = true;
multiplots[1,0].LogY = true;
multiplots[1,0].tmargin = 0;
multiplots[1,0].bmargin = 3;
multiplots[1,0].XrangeMin=xMin;
multiplots[1,0].XrangeMax=xMax;
multiplots[1,0].Xlabel = "no of cores";
multiplots[1,0].Ylabel = "iterations";
multiplots[1,0].LegendAlignment=new string[]{"i","b","r"};
multiplots.PlotNow()

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside bottom right Left reverse 
set key font ",16"inside bottom right Left reverse 
Gnuplot Error: Warning: empty y range [26:26], adjusting to [25.74:26.26]



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 w a llclock time 
 
 
 
 
 Kcycle w. add.-Schwarz DG2 
 
 
 
 
 Kcycle w. add.-Schwarz DG2 
 
 
 
 
 
 
 
 2h limit 
 
 
 
 
 2h limit 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 iterations 
 
 
 
 
 no of cores 
 
 
 
 
 Kcycle w. add.-Schwarz DG2 
 
 
 
 
 Kcycle w. add.-Schwarz DG2

In [ ]:
// ==============
// plot out stuff
// ==============
string plotsdirpath = CurrentDocDir+@"\"+"plots";
DirectoryInfo dir   = Directory.CreateDirectory(plotsdirpath);
var plot2print      = multiplots;
var CL              = plot2print.ToGnuplot().PlotCairolatex(xSize:14,ySize:12);
CL.WriteMinimalCompileableExample(
   Path.Combine(plotsdirpath, "Scaling_"+DG+"_"+".tex"),
   "Scaling_"+DG+".tex");

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside bottom right Left reverse 
set key font ",16"inside bottom right Left reverse 
Gnuplot Error: Warning: empty y range [26:26], adjusting to [25.74:26.26]

